In [7]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from glob import glob
import json
import wave
import math
from itertools import permutations

import numpy as np
from scipy import stats, signal

import essentia.standard as es
import librosa

import tensorflow as tf
from model.fp.melspec.melspectrogram import get_melspec_layer, Melspec_layer_essentia
from run import load_config
from model.utils.audio_utils import load_audio, background_mix, max_normalize, ir_aug

import IPython.display as ipd

from model.dataset import Dataset

In [2]:
DISCOTUBE = "/mnt/projects/discotube/"
discotube_2020 = DISCOTUBE + "discotube-2020-09/audio"
discotube_2023 = DISCOTUBE + "discotube-2023-03/audio-new/audio/"

# discotube_2023_mp4_paths = glob(os.path.join(discotube_2023, "**/*.mp4"), recursive=True)
# print(len(discotube_2023_mp4_paths))
# discotube_2020_mp4_paths = glob(os.path.join(discotube_2020, "**/*.mp4"), recursive=True)
# print(len(discotube_2020_mp4_paths))

In [14]:
with open("/home/oaraz/nextcore/afp/discotube-2023.txt", "r") as in_f, open("/home/oaraz/nextcore/afp/discotube-2023.txt.filtered", "w") as out_f:
    for mp4_path in in_f.readlines():
        meta_path = mp4_path[:-4]+"meta"
        with open(meta_path, "r") as meta_f:
            metadata = json.load(meta_f)
        if metadata["duration"] < 420:
            out_f.write(mp4_path)
        break

In [12]:
type(metadata["duration"])

int

In [13]:
metadata["id"]

'F_oWcMGPn3I'

In [10]:
list(metadata.keys())

['id',
 'title',
 'formats',
 'thumbnails',
 'thumbnail',
 'description',
 'uploader',
 'uploader_id',
 'uploader_url',
 'channel_id',
 'channel_url',
 'duration',
 'view_count',
 'average_rating',
 'age_limit',
 'webpage_url',
 'categories',
 'tags',
 'playable_in_embed',
 'live_status',
 'release_timestamp',
 '_format_sort_fields',
 'automatic_captions',
 'subtitles',
 'album',
 'artist',
 'track',
 'release_date',
 'release_year',
 'comment_count',
 'chapters',
 'like_count',
 'channel',
 'channel_follower_count',
 'upload_date',
 'creator',
 'alt_title',
 'availability',
 'original_url',
 'webpage_url_basename',
 'webpage_url_domain',
 'extractor',
 'extractor_key',
 'playlist',
 'playlist_index',
 'display_id',
 'fulltitle',
 'duration_string',
 'is_live',
 'was_live',
 'requested_subtitles',
 '_has_drm',
 'asr',
 'filesize',
 'format_id',
 'format_note',
 'source_preference',
 'fps',
 'audio_channels',
 'height',
 'quality',
 'has_drm',
 'tbr',
 'url',
 'width',
 'language',
 'la

In [20]:
cfg_path = "default"
cfg = load_config(cfg_path)

cli: Configuration from ./config/default.yaml


In [21]:
dataset = Dataset(cfg)
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])
val_ds = dataset.get_val_ds() # max 500

In [25]:
train_ds.__getitem__(1000)

(array([[-0.15350342, -0.11679077, -0.09753418, ..., -0.03204346,
         -0.1595459 , -0.22732544],
        [-0.5508728 , -0.56887817, -0.64001465, ...,  0.39117432,
          0.61026001,  0.70654297],
        [ 0.00942993,  0.00344849,  0.01669312, ...,  0.02722168,
          0.02902222,  0.02914429],
        ...,
        [ 0.22784424,  0.17156982,  0.1572876 , ..., -0.1428833 ,
         -0.30743408, -0.32946777],
        [ 0.03091431,  0.02310181,  0.00543213, ..., -0.01507568,
         -0.02331543, -0.02084351],
        [ 0.02630615,  0.01245117,  0.03186035, ...,  0.13760376,
          0.14871216,  0.13946533]]),
 array([[-0.25218626, -0.27093921, -0.16075578, ..., -0.15305549,
          0.06177466, -0.01334165],
        [ 0.13872359,  0.04715587, -0.20875533, ...,  0.00538362,
          0.02341372,  0.1252297 ],
        [-0.12350705, -0.04640386, -0.16751183, ...,  0.2849225 ,
          0.49900552,  0.24563858],
        ...,
        [-0.19791371, -0.23189545, -0.20542681, ..., -

In [11]:
for fname, seg_idx, offset_min, offset_max in train_ds.fns_event_seg_list:

    if os.path.basename(fname) == '114558.wav':
    #if offset_max==0:# and offset_max==0:
        print(fname)
        print(seg_idx)
        print(offset_min)
        print(offset_max)
        print()

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
0
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
1
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
2
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
3
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
4
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
5
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
6
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
7
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
8
-4000
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma

In [8]:
len(train_ds.fns_event_seg_list)

581850

In [15]:
np.max([-4000, -train_ds.offset_margin_frame])

-1600

In [16]:
np.min([0.0, train_ds.offset_margin_frame])

0.0

In [20]:
for fname, seg_idx, offset_min, offset_max in train_ds.fns_event_seg_list:

    if os.path.basename(fname) == '114558.wav':
    #if offset_max==0:# and offset_max==0:
        print(fname)
        print(seg_idx)
        print(offset_min)
        print(offset_max)
        print()

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
0
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
1
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
2
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
3
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
4
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
5
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
6
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
7
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114558.wav
8
0
4000.0

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/114/114